In [4]:
%config IPCompleter.greedy=True

In [5]:
import pandas as pd
import numpy as np

checkin = pd.read_csv("./Datasets/dataset_TIST2015/smallCheckin.csv", header=None, sep="\t")
cities = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_Cities.txt", header=None, sep="\t")
pois = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_POIs.txt", header=None, sep="\t")

checkin.columns = ['user_id','venue_id','utc_time','offset']
pois.columns = ['venue_id','latitude','longitude','category','country_code']
cities.columns = ['city','latitude','longitude','country_code','country_name','city_type']

df1 = checkin.merge(pois, on="venue_id")


In [6]:
v = set()
size = 60
def chunk_preprocess(df,field='user_id'):
    global v,size
    df.columns= ['user_id','venue_id','utc_time','offset']
    uslen = len(df['user_id'].unique())
    #print('No. of users: ', uslen)
    if size > len(v):
        a = 0
        if uslen > size : a = size
        else : a = uslen
        v = v|set(random.sample(list(df['user_id'].unique()), a))
        print('sss ',size,' ',len(v))
    d = df[df[field].isin(v)]
    return d

In [7]:
import random
v = set()
size =1000

In [8]:
checkin_chunk = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_Checkins.txt", header=None,sep="\t",chunksize = 150000)
checkins_list = []
for i in checkin_chunk:
    df  = chunk_preprocess(i)
    if df is not None: 
        checkins_list.append(df)
        #print('subframe len',len(df.index))
        #print('No. of users: ', len(df['user_id'].unique()))
        # print(len(checkins_list))
    else:
        break
if len(checkins_list) >= 1: checkindf = pd.concat(checkins_list)
else: print("000000000000")
print('final len', len(checkindf.index))
print('No. of users: ', len(checkindf['user_id'].unique()))

sss  1000   1000
final len 201050
No. of users:  1000


In [9]:
checkindf.to_csv("./Datasets/dataset_TIST2015/smalldata.csv", index=False)

In [10]:
c = pd.read_csv("./Datasets/dataset_TIST2015/smalldata.csv")

In [11]:

c = c.drop('offset', axis=1)
c.to_csv("./Datasets/dataset_TIST2015/smalldata.csv", index=False)

In [12]:
c.head()

,user_id,venue_id,utc_time
0,3861,4e610977ae60e9edf521a9b4,Tue Apr 03 18:00:52 +0000 2012
1,174163,4b6f2cf8f964a52013e22ce3,Tue Apr 03 18:00:55 +0000 2012
2,174163,4d99f4252bd6f04d2c138a50,Tue Apr 03 18:01:35 +0000 2012
3,155775,4f74047de4b0f112e0fb1bb9,Tue Apr 03 18:01:39 +0000 2012
4,10339,4d30438782fd5481b758b1a8,Tue Apr 03 18:01:45 +0000 2012


In [13]:
len(c.index)

201050

In [14]:
c['utc_time'] = pd.to_datetime(c['utc_time'])


In [15]:
c['utc_time'] = c['utc_time'] - c['utc_time'].min()
c['utc_time'] = c['utc_time'].dt.seconds


In [16]:
c.head()

,user_id,venue_id,utc_time
0,3861,4e610977ae60e9edf521a9b4,0
1,174163,4b6f2cf8f964a52013e22ce3,3
2,174163,4d99f4252bd6f04d2c138a50,43
3,155775,4f74047de4b0f112e0fb1bb9,47
4,10339,4d30438782fd5481b758b1a8,53


In [17]:
df= c.merge(pois,on="venue_id")


In [18]:
len(df.index)

201050

In [19]:
df = df.drop(['latitude','longitude'], axis=1)
df.head()
len(np.unique(df['category']))


422

In [20]:
d = {k:v for v,k in enumerate(list(set(df['category'])))}
print('No. of cats: ', len(d.keys()))
df['category'] = df['category'].map(d)

No. of cats:  422


In [21]:
df.head()

,user_id,venue_id,utc_time,category,country_code
0,3861,4e610977ae60e9edf521a9b4,0,337,TR
1,174163,4b6f2cf8f964a52013e22ce3,3,274,CL
2,13643,4b6f2cf8f964a52013e22ce3,4327,274,CL
3,6322,4b6f2cf8f964a52013e22ce3,1483,274,CL
4,23940,4b6f2cf8f964a52013e22ce3,2916,274,CL


In [22]:
d = {k:v for v,k in enumerate(list(set(df['country_code'])))}
print('No. of countries: ', len(d.keys()))
df['country_code'] = df['country_code'].map(d)

No. of countries:  68


In [23]:
df.head()

,user_id,venue_id,utc_time,category,country_code
0,3861,4e610977ae60e9edf521a9b4,0,337,6
1,174163,4b6f2cf8f964a52013e22ce3,3,274,0
2,13643,4b6f2cf8f964a52013e22ce3,4327,274,0
3,6322,4b6f2cf8f964a52013e22ce3,1483,274,0
4,23940,4b6f2cf8f964a52013e22ce3,2916,274,0


In [24]:
d = {k:v for v,k in enumerate(list(set(df['venue_id'])))}
print('No. of venues: ', len(d.keys()))
df['venue_id'] = df['venue_id'].map(d)

No. of venues:  70419


In [25]:
df.head()

,user_id,venue_id,utc_time,category,country_code
0,3861,41105,0,337,6
1,174163,15523,3,274,0
2,13643,15523,4327,274,0
3,6322,15523,1483,274,0
4,23940,15523,2916,274,0


In [26]:
len(set(df['user_id']))

1000

In [27]:
df.sort_values(by=['user_id','utc_time'],inplace=True)
df.to_csv("./Datasets/dataset_TIST2015/smalldata.csv", index=False, sep='\t')

In [28]:
df.head()

,user_id,venue_id,utc_time,category,country_code
94710,26,39031,1,52,53
61512,26,133,496,305,53
171427,26,25628,7101,376,53
142790,26,63884,7614,305,53
114979,26,30471,7665,305,53
